<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Turkish/FastText_biGRU_crossval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Turkish Language

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [50]:
!pip install focal-loss

In [2]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Turkish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Turkish


In [3]:
ls

cc.tr.300.vec     offenseval-annotation.txt      readme-trainingset-tr.txt
cc.tr.300.vec.gz  offenseval-tr-training-v1.tsv


In [0]:
import pandas as pd
train=pd.read_csv( 'offenseval-tr-training-v1.tsv',sep="\t")
train=train.sample(frac=1)

In [5]:
train.head()

,id,tweet,subtask_a
29507,38775,Bi gün yemek yapmıştım babam beğenmedi ve yeme...,NOT
8612,43317,Poğaçacı abi hesabı ödedikden sonra; İyi bir h...,NOT
8195,26839,#TaksicilerAçıklamaBekliyor Uberi yasaklayın y...,OFF
9580,24562,"“ Bir çiçek bahçesinde geceye durgun kalışın, ...",NOT
3788,45773,"Bugün Senin İçin Bir Şey Yapmıyorsam, Dün S...",NOT


In [6]:
train.tail()

,id,tweet,subtask_a
27194,37520,@USER ~ilk hissettiklerim:bir jichu gördüm san...,NOT
22221,39051,güzel olan her şey şeytani,NOT
1014,45084,@USER Canım benim görende 4 te 4 yaptınız sanı...,NOT
21307,39583,@USER Hayatı zorlaştıran böyle şeyler galiba,NOT
611,18778,Ayrıca Atatürk siyaset üstüdür. Hiçbir siyasiy...,NOT


In [0]:
labels=train['subtask_a']
train=train['tweet']
labels=pd.factorize(labels)[0]

In [8]:
import collections
collections.Counter(labels)

Counter({0: 25231, 1: 6046})

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [10]:
len(sequences_matrix),len(labels)

(31277, 31277)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
import numpy as np
from sklearn.model_selection import StratifiedKFold
from focal_loss import BinaryFocalLoss
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [13]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

10001


In [0]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz

In [0]:
# import gzip
# import shutil
# with gzip.open('cc.tr.300.vec.gz', 'rb') as f_in:
#     with open('cc.tr.300.vec', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
embeddings=build_matrix('cc.tr.300.vec', word_index)


In [18]:
embeddings.shape

(10001, 300)

In [0]:
def model_func():
  model = Sequential()
  model.add(Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False))
  model.add(Bidirectional(CuDNNGRU(50, return_sequences = True)))
  #model.add(Bidirectional(CuDNNGRU(32, return_sequences = True)))
  model.add(GlobalMaxPool1D())
  model.add(Dense(50, activation="relu"))
  model.add(Dense(20, activation="relu"))
  model.add(Dense(10, activation="relu"))

  model.add(Dropout(0.05))
  model.add(Dense(1, activation="sigmoid"))
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [55]:
kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=2020)
cvscores = []

for train, test in kfold.split(sequences_matrix, labels):
  model=model_func()
  
  model.fit(sequences_matrix[train], labels[train],batch_size=256,epochs=10,verbose=2,class_weight=class_weights,
            validation_data=(sequences_matrix[test],labels[test]),callbacks=[reduce_lr])
  y_pred = model.predict(sequences_matrix[test], batch_size=128, verbose=1)
  y_pred_bool = np.argmax(y_pred, axis=1)
  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[test], y_pred, average='macro')
  cvscores.append(f1)
  keras.backend.clear_session()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Train on 28149 samples, validate on 3128 samples
Epoch 1/10
 - 4s - loss: 0.1242 - acc: 0.8108 - val_loss: 0.1007 - val_acc: 0.8465
Epoch 2/10
 - 4s - loss: 0.1007 - acc: 0.8468 - val_loss: 0.0949 - val_acc: 0.8597
Epoch 3/10
 - 4s - loss: 0.0951 - acc: 0.8580 - val_loss: 0.0935 - val_acc: 0.8616
Epoch 4/10
 - 4s - loss: 0.0923 - acc: 0.8624 - val_loss: 0.0906 - val_acc: 0.8696
Epoch 5/10
 - 4s - loss: 0.0890 - acc: 0.8671 - val_loss: 0.0894 - val_acc: 0.8680
Epoch 6/10
 - 4s - loss: 0.0866 - acc: 0.8710 - val_loss: 0.0891 - val_acc: 0.8667
Epoch 7/10
 - 4s - loss: 0.0825 - acc: 0.8782 - val_loss: 0.0899 - val_acc: 0.8645
Epoch 8/10
 - 4s - loss: 0.0800 - acc: 0.8791 - val_loss: 0.0933 - val_acc: 0.8638

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 9/10
 - 4s - loss: 0.0707 - acc: 0.8969 - val_loss: 0.0915 - val_acc: 0.8657
Epoch 10/10
 - 4s - loss: 0.0681 - acc: 0.9006 - val_loss: 0.0930 - val_acc: 0.8648

Epoch 00010: ReduceLROnPlateau reduci

In [56]:
np.array(cvscores).mean()

0.7360764729687974